# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [1]:
# Install Required Packages
%pip install optiguide

Note: you may need to restart the kernel to use updated packages.


In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [3]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/niharikp/football_players_transfers_optimisation/main/model_script.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from gurobipy import Model, GRB, quicksum
import pandas as pd

# Cleaned dataset with pre-calculated performance metric
data_path = 'https://raw.githubusercontent.com/gmoellering/dabp/main/final_merged.csv' #Replace data path here
players_data = pd.read_csv(data_path)

#Constraint variables
team_var = '131' #FC Barcelona
budget_var = 150000000
.
.
.

            team_player_dict[team_id] = [player_id]
       


for team in team_player_dict.keys():
    print(team)
    selected_players = players_data[players_data['player_id'].isin(team_player_dict[team])]
    print(selected_players[['player_id', 'name', 'position', 'normalised_performance_score', 'market_value_in_eur']])




In [5]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
%%capture
agent = OptiGuideAgent(
    name="OptiGuide Football Example",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create our user questions.

---

#### Question 1

With this question, we attempted to get a baseline result from our model that could act as a springboard for future questions and/or adjustments. We wanted it to provide a description that included the players in the optimal solution and at least some justification for why those players were selected over the others. 

In [7]:
user.initiate_chat(agent, message="What is our optimal player lineup if we want to trade for at least one attack position, one defender position, and one midfield position and why should we select those players?")

user (to OptiGuide Football Example):

What is our optimal player lineup if we want to trade for at least one attack position, one defender position, and one midfield position and why should we select those players?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# The code to answer the question is actually already in place in the last part
# of the provided code. It runs the model given the constraints, finds the optimal solution,
# and then prints out the players selected for the team and their details. The code is as follows:

# Initialize dictionary for selected players
team_player_dict = {}

# Iterate over model variables
for v in model.getVars():
    # If the player is chosen in the solution
    if v.x > 0:
        # Extracting i and j from the variable name
 

As we can see, the model successfully accommodates our constraint to include at least one player of each position type. The model appears to have chosen players that offer a good balance of market value and overall performance. However, the model is unable to take into account aspects of personal preferences, team dynamics and player health. This, essentially, is one of the model’s primary limitations, as such qualitative identifiers are hard to track or numerically inculcate into the model itself. One thing we were disappointed with was that the OptiGuide response only created a placeholder for the actual player data returned from code instead of inserting it into its output on its own.


---

#### Question 2

This question was designed to assess our model’s ability to take into account intangible factors like team dynamics. The use case for such a question would be if a coach, manager, or other stakeholder saw one of the players returned by the optimization and immediately ruled them out due to an intangible factor like propensity for conflict with existing players, behavioral problems off the field, the player having previously expressed having no interest playing for the team, etc. 

In [8]:
user.initiate_chat(agent, message="We do not like player Sergio Busquets, who should we hire if we remove him from consideration?")

user (to OptiGuide Football Example):

We do not like player Sergio Busquets, who should we hire if we remove him from consideration?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Define the player_id of Sergio Busquets, which can be found in your player database.
sergio_busquets_id = 0  # replace it with the real id

# Exclude Sergio Busquets
players_to_be_excluded.append(sergio_busquets_id)
# Exclude team player
if players_to_be_excluded:
    players_data = players_data[~players_data['player_id'].isin(players_to_be_excluded)]

# Run the model again and find the optimal solution
model.optimize()
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Define the player_id of

Although our output looks intelligible based on the human readable answer, the optimization results show that Sergio is still in the selected line-up. If OptiGuide is having trouble interpreting names, we'll try again using a different player (since it's comparing results to previous responses) and specifically referencing his player_id.

In [9]:
user.initiate_chat(agent, message="We do not like player 15452, who should we hire if we remove him from consideration?")

user (to OptiGuide Football Example):

We do not like player 15452, who should we hire if we remove him from consideration?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Add the player to the exclusion list
players_to_be_excluded.append(15452)

# Remove the player from the dataset
players_data = players_data[~players_data['player_id'].isin(players_to_be_excluded)]
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Add the player to the exclusion list
players_to_be_excluded.append(15452)

# Remove the player from the dataset
players_data = players_data[~players_data['player_id'].isin(players_to_be_excluded)]

--- One-Word Answer: SAFE or DANGER ---


--------------------

This time the optimized results did not include player 15452 (Raúl Albiol), so it accomplished what we asked it to do.


---

#### Question 3

We chose Lewandowski specifically because he had the most goals in 2022, and we wanted to see how the model would adjust when forced to incorporate his relatively high market value. 

In [10]:
user.initiate_chat(agent, message="We definitely want player 38253 who else should we hire if he absolutely has to be in the lineup?")

user (to OptiGuide Football Example):

We definitely want player 38253 who else should we hire if he absolutely has to be in the lineup?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
player_to_be_included.append(38253)
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
player_to_be_included.append(38253)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Football Example):

SAFE

--------------------------------------------------------------------------------
unsupported operand type(s) for -: 'dict' and 'float'
OptiGuide Football Example (to writer):



While running the code y

This output is deceptive because 38253 is not actually in the optimized solution. (Note: Our initial output disappeared when we re-ran the notebook, but the results we refer to in this comment are included in the report)

In [11]:
user.initiate_chat(agent, message="We definitely want player_id 38253, Robert Lewandowski, who else should we hire if he absolutely has to be in the lineup?")

user (to OptiGuide Football Example):

We definitely want player_id 38253, Robert Lewandowski, who else should we hire if he absolutely has to be in the lineup?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Include Robert Lewandowski, player_id 38253 in the lineup,
# we update the player_to_be_included list as follows:
player_to_be_included = [38253]
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Include Robert Lewandowski, player_id 38253 in the lineup,
# we update the player_to_be_included list as follows:
player_to_be_included = [38253]

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safe

Rephrasing the question this time led to a "Sorry. I cannot answer your question." response.

In [12]:
user.initiate_chat(agent, message="We definitely want Robert Lewandowski, who else should we hire if he absolutely has to be in the lineup?")

user (to OptiGuide Football Example):

We definitely want Robert Lewandowski, who else should we hire if he absolutely has to be in the lineup?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Robert Lewandowski's player_id
lewandowski_id = players_data[players_data['name'] == 'Robert Lewandowski']['player_id'].iloc[0]
player_to_be_included.append(lewandowski_id)
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Robert Lewandowski's player_id
lewandowski_id = players_data[players_data['name'] == 'Robert Lewandowski']['player_id'].iloc[0]
player_to_be_included.append(lewandowski_id)

--- One-Word Answer: SAFE or DANGER ---


------------------------------------------------

This rephrasing was not successful either.

In [13]:
user.initiate_chat(agent, message="Robert Lewandowski, player_id 38253 has to be in the optimal solution, what other players should we select with him?")

user (to OptiGuide Football Example):

Robert Lewandowski, player_id 38253 has to be in the optimal solution, what other players should we select with him?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Adding Robert Lewandowski to the model and adjusting the budget and players needed
player_to_be_included.append(38253)
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Adding Robert Lewandowski to the model and adjusting the budget and players needed
player_to_be_included.append(38253)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Football Example):

SAFE

-------------

---

#### Question 4

In this question, we wanted to see what our model would do when we tightened the budget constraint and if that would lead to meaningful changes in our original line-up. This question gave us some problems as well. Again, the language in the output in our first attempt implied that OptiGuide understood what we were asking but the output in the code didn’t line up with its response.

In [14]:
user.initiate_chat(agent, message="Our budget decreased by 5 million euros, how does that change which players we should try to trade for?")

user (to OptiGuide Football Example):

Our budget decreased by 5 million euros, how does that change which players we should try to trade for?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Decrease the budget for the selected team
BUDGET[team_var] -= 5000000

# Update the budget constraint
for constr in model.getConstrs():
    if constr.ConstrName == 'Budget_{}'.format(team_var):
        model.remove(constr)
model.addConstr(quicksum(players_data.loc[players_data['player_id'] == i, 'market_value_in_eur'].iloc[0] * x[(i, team_var)]
                             for i in players_data['player_id']) <= BUDGET[team_var], name='Budget_{}'.format(team_var))

# Solve again
model.optimize()

# Print the players again
team_player_dict = {}
for v in model.getVars():
    if v.

We see that the output looks good initially, but that the selected player market values actually add up to 154 million, not 145 million. 

In [15]:
user.initiate_chat(agent, message="Our budget is now 145 million euros, how does that change which players we should try to trade for?")

user (to OptiGuide Football Example):

Our budget is now 145 million euros, how does that change which players we should try to trade for?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Update the budget
BUDGET[team_var] = 145000000

# Update the model to reflect the new budget and re-optimize
model.update()
model.optimize()
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Update the budget
BUDGET[team_var] = 145000000

# Update the model to reflect the new budget and re-optimize
model.update()
model.optimize()

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Football Exa

Massaging the question did not help us get to the correct solution.

---

#### Question 5

In this question, we wanted to see what our model would do when we relaxed the budget constraint and if that would lead to meaningful changes in our original line-up. This question held up well when we ran it multiple times - the verbiage in the responses changed slightly but the output was the same.

In [19]:
user.initiate_chat(agent, message="Our budget increased by 10 million euros, how does that change which players we should try to trade for?")

user (to OptiGuide Football Example):

Our budget increased by 10 million euros, how does that change which players we should try to trade for?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
#Increasing the budget by 10 million euros
BUDGET[team_var] += 10000000
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
#Increasing the budget by 10 million euros
BUDGET[team_var] += 10000000

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Football Example):

SAFE

--------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.0 build v10.